# Select

Selecting data is the most important aspect of SQL for database users. This page provides an overview of different statements that can be used in the `SELECT` expression.

We'll use PostgreSQL as the standard; other databases will be considered in specific sections.

In [1]:
docker run -d --rm\
    --name select_examples\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

**Note**: Don't forget to stop the container when finished.

In [10]:
docker stop select_examples

select_examples


## Statements order

You must remember that SQL statements are executed in a specific order. The following picture shows the order of basic sql statements.

![order](select_files/order_picture.png)

Origin of the image is [this page](https://www.kdnuggets.com/the-essential-guide-to-sql-execution-order).

## Sorting (ORDER BY)

Result of the query can be sorted using syntax `ORDER BY <columns> (DESC)`. Find out more in the [special page](select/sorting.ipynb).

---

The following cell demonstrates an example where the query uses `ORDER BY numeric_value`, ensuring that the entire table follows this order.

In [3]:
docker exec -i select_examples psql -U postgres -d postgres << EOF
SELECT *
FROM (
    VALUES (7, 'a'), (5, 'b'), (1, 'c'), (4, 'd'), (4, 'e'), (5, 'f')
) AS temp(numeric_value, symbol)
ORDER BY numeric_value;
EOF

 numeric_value | symbol 
---------------+--------
             1 | c
             4 | d
             4 | e
             5 | b
             5 | f
             7 | a
(6 rows)



## IS (NOT) NULL

You can check if value (not) taking NULL value using following suntax `<value> IS (NOT) NULL`.

---

The following example shows the original column and the application of `IS NULL` and `IS NOT NULL` to it.

In [15]:
docker exec -i select_examples psql -U postgres -d postgres << EOF
SELECT
    numeric_value,
    numeric_value IS NULL is_null,
    numeric_value IS NOT NULL is_not_null
FROM (
    VALUES (NULL), (1), (2), (NULL), (4), (5)
) AS temp(numeric_value);
EOF

 numeric_value | is_null | is_not_null 
---------------+---------+-------------
               | t       | f
             1 | f       | t
             2 | f       | t
               | t       | f
             4 | f       | t
             5 | f       | t
(6 rows)



## Conditional (CASE)

With the `CASE` keyword, you can assign specified values to logical conditions. Find out more in the [specific page](select/conditional.ipynb).

---

The following cell shows how you can assign a string that describes the value originally in the column.

In [9]:
docker exec -i select_examples psql -U postgres -d postgres << EOF
SELECT
    numeric_value,
    CASE
        WHEN numeric_value<2 THEN 'less than two'
        ELSE 'more or equals than two'
    END
FROM (
    VALUES (0), (1), (2), (3), (4), (5)
) AS temp(numeric_value);
EOF

 numeric_value |          case           
---------------+-------------------------
             0 | less than two
             1 | less than two
             2 | more or equals than two
             3 | more or equals than two
             4 | more or equals than two
             5 | more or equals than two
(6 rows)



## Empty values

Dealing with empty values is a typical task when working with databases. Find out more in a [specific page](select/empty_values.ipynb).

## Array expanding

If you have columns of arrays and need to create a separate record for each value in the array, you may need operations to unnest the arrays. Find out more in [particular page](select/expand_array.ipynb).

---

The following cell demonstrates the result of applying the `unnest` function to a column containing arrays. Each array is expanded so that each element is represented in a separate record, duplicating the other columns' values accordingly.

In [2]:
docker exec -i select_examples psql -U postgres << EOF

SELECT 
    col1, 
    unnest(col2) AS unnestted_value
FROM (
    VALUES
    (10, '{1,2,3}'::int[]),
    (7, '{3,4,1}'::int[])
) AS temp(col1, col2);

EOF

 col1 | unnestted_value 
------+-----------------
   10 |               1
   10 |               2
   10 |               3
    7 |               3
    7 |               4
    7 |               1
(6 rows)

